In [1]:
# all required imports
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from geopy.distance import geodesic
import json
import os
import math
import sys
import math
import random
import pickle
import datetime
import scipy as sp
from collections import OrderedDict
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [2]:
print(os.listdir("."))

['.DS_Store', 'create-venues-data.ipynb', 'RankingAcademicPapers.ipynb', 'authors-keywords.ipynb', '.ipynb_checkpoints', '.git', 'data', 'get-author-domains.ipynb']


In [3]:
print(os.listdir("./data/"))

['aminer_papers_11.txt', 'aminer_papers_1.txt', 'aminer_papers_0.txt', 'aminer_papers_10.txt', 'venues.csv', 'aminer_papers_12.txt', 'aminer_papers_2.txt', 'aminer_papers_3.txt', 'aminer_papers_13.txt', 'authors_df_sorted_by_num_citations_all.csv', 'aminer_papers_17.txt', 'aminer_papers_7.txt', 'aminer_papers_6.txt', 'aminer_papers_16.txt', 'aminer_papers_14.txt', 'aminer_papers_28.txt', 'aminer_papers_4.txt', 'aminer_papers_5.txt', 'aminer_papers_29.txt', 'aminer_papers_15.txt', 'auth_by_name.pkl', 'authors.csv', 'authors_with_keywords.csv', 'dblp-ref', 'AMiner-Author.txt', 'authors_df_sorted_by_num_citations.csv', 'aminer_papers_18.txt', 'aminer_papers_24.txt', 'aminer_papers_8.txt', 'aminer_papers_9.txt', 'aminer_papers_25.txt', 'aminer_papers_19.txt', 'linking_relations.txt.zip', 'authors_sorted_by_h_index.csv', 'aminer_papers_27.txt', 'aminer_papers_26.txt', 'aminer_papers_22.txt', 'aminer_papers_23.txt', 'authors_sorted_by_h_index_all.csv', 'aminer_papers_21.txt', 'aminer_papers_

In [4]:
authors_df = pd.read_csv('./data/authors_with_keywords.csv')

In [5]:
authors_df.head(10)

,Unnamed: 0,author_id,citations,co_authors,h_index,num_citations,paper_count,keywords,affiliation
0,Makoto_Satoh,1,102,"[2, 3, 4, 5, 6, 7, 8, 40149, 463789, 463790, 4...",2,24.142857,5,middle school science classroom;science discou...,"Georgia Institute of Technology, Atlanta, GA"
1,Ryo_Muramatsu,2,0,"[1, 3, 4, 5, 6, 7, 8]",0,0.000000,1,NaN,NaN
2,Mizue_Kayama,3,655,"[1, 2, 4, 5, 6, 7, 8, 73800, 30142, 198087, 49...",13,213.900000,32,collaborative learning;individual learning;sup...,"Senshu University;Shinshu University, Faculty ..."
3,Kazunori_Itoh,4,364,"[1, 2, 3, 5, 6, 7, 8, 73800, 201615, 334037, 4...",8,93.350000,20,localized sound;movement sense;pattern represe...,"DIINF, University of Santiago of Chile, Chile"
4,Masami_Hashimoto,5,153,"[1, 2, 3, 4, 6, 7, 8, 201615, 490012, 509990, ...",3,23.100000,10,chorus leader;Tutoring Knowledge;support syste...,"Faculty of Engineering, Shinshu University, Na..."
5,Makoto_Otani,6,483,"[1, 2, 3, 4, 5, 7, 8, 50209, 324798, 180194, 2...",9,98.450000,39,chorus leader;virtual sound space;Tutoring Kno...,"Toyama Prefectural University, Department of I..."
6,Michio_Shimizu,7,260,"[1, 2, 3, 4, 5, 6, 8, 201615, 334037, 509990, ...",6,65.833333,10,digital binary pattern;digital figure;segments...,Centro de Investigación Avanzada en Ingeniería...
7,Masahiko_Sugimoto,8,200,"[1, 2, 3, 4, 5, 6, 7, 509990, 509991, 419420, ...",4,45.833333,7,Othello game;pen input guide;auditory sense;co...,"Graduate School of Science and Technology, Shi..."
8,Gareth_Beale,9,110,"[10, 380963, 380964, 380965, 337260, 178003, 7...",3,31.984127,4,Roman statue head;accurate visualisation;case ...,"University of Southampton, UK"
9,Graeme_Earl,10,565,"[9, 323945, 446287, 446288, 109746, 569693, 56...",8,108.498016,9,Minoan cemetery;RTI datasets;RTI practice;anci...,"University of Southampton, Archaeological Comp..."


In [6]:
authors_df.head(100)

,Unnamed: 0,author_id,citations,co_authors,h_index,num_citations,paper_count,keywords,affiliation
0,Makoto_Satoh,1,102,"[2, 3, 4, 5, 6, 7, 8, 40149, 463789, 463790, 4...",2,24.142857,5,middle school science classroom;science discou...,"Georgia Institute of Technology, Atlanta, GA"
1,Ryo_Muramatsu,2,0,"[1, 3, 4, 5, 6, 7, 8]",0,0.000000,1,NaN,NaN
2,Mizue_Kayama,3,655,"[1, 2, 4, 5, 6, 7, 8, 73800, 30142, 198087, 49...",13,213.900000,32,collaborative learning;individual learning;sup...,"Senshu University;Shinshu University, Faculty ..."
3,Kazunori_Itoh,4,364,"[1, 2, 3, 5, 6, 7, 8, 73800, 201615, 334037, 4...",8,93.350000,20,localized sound;movement sense;pattern represe...,"DIINF, University of Santiago of Chile, Chile"
4,Masami_Hashimoto,5,153,"[1, 2, 3, 4, 6, 7, 8, 201615, 490012, 509990, ...",3,23.100000,10,chorus leader;Tutoring Knowledge;support syste...,"Faculty of Engineering, Shinshu University, Na..."
5,Makoto_Otani,6,483,"[1, 2, 3, 4, 5, 7, 8, 50209, 324798, 180194, 2...",9,98.450000,39,chorus leader;virtual sound space;Tutoring Kno...,"Toyama Prefectural University, Department of I..."
6,Michio_Shimizu,7,260,"[1, 2, 3, 4, 5, 6, 8, 201615, 334037, 509990, ...",6,65.833333,10,digital binary pattern;digital figure;segments...,Centro de Investigación Avanzada en Ingeniería...
7,Masahiko_Sugimoto,8,200,"[1, 2, 3, 4, 5, 6, 7, 509990, 509991, 419420, ...",4,45.833333,7,Othello game;pen input guide;auditory sense;co...,"Graduate School of Science and Technology, Shi..."
8,Gareth_Beale,9,110,"[10, 380963, 380964, 380965, 337260, 178003, 7...",3,31.984127,4,Roman statue head;accurate visualisation;case ...,"University of Southampton, UK"
9,Graeme_Earl,10,565,"[9, 323945, 446287, 446288, 109746, 569693, 56...",8,108.498016,9,Minoan cemetery;RTI datasets;RTI practice;anci...,"University of Southampton, Archaeological Comp..."


In [47]:
author_keyword_map = {}
co_authors_map = {}
for index, row in authors_df.iterrows():
    if not pd.isna(row['co_authors']):
#         co_authors = set(row['co_authors'].lstrip("[").rstrip("]").replace(" ", "").split(","))
        co_authors = set(map(int, row['co_authors'].lstrip("[").rstrip("]").replace(" ", "").split(",")))
        co_authors_map[row['author_id']] = co_authors
    if not pd.isna(row['keywords']):
        keywords = set(row['keywords'].split(';'))
        author_keyword_map[row['author_id']] = set(keywords)

In [48]:
#random.sample(set([1, 2, 3, 4, 5, 6]), 6)
# print(co_authors_map[2])
authors_df = authors_df.sort_values('h_index',ascending=False)

In [53]:

idx = 0
bucket_map = {}
author_bucket_map = {}
count_buckets = 0
for index, row in authors_df.iterrows():
    if idx==100000:
        break
    idx+=1
    author_id = row['author_id']
    max_intersect_len = 0
    count_max = 0
    intersecting_sets = []
    if author_id not in author_keyword_map:
        if author_id not in co_authors_map:
            continue
        author_keyword_set = set()
        co_authors_set = co_authors_map[author_id]
        for co_author_id in co_authors_set:
            if co_author_id in author_keyword_map:
                author_keyword_set = author_keyword_set | author_keyword_map[co_author_id]
        keywords_list = random.sample(author_keyword_set, min(len(author_keyword_set), 10))
        author_keyword_map[author_id] = set(keywords_list)
    if author_id in author_keyword_map:
        keywords_set = author_keyword_map[author_id]
        for key in bucket_map:
#             print(author_id, key, keywords_set, bucket_map[key])
#             print(author_id, author_keyword_map[author_id], type(keywords_set))
            len_intersection = len(keywords_set.intersection(bucket_map[key]))
            if len_intersection>max_intersect_len:
                max_intersect_len = len_intersection
                count_max = 1
                intersecting_sets = []
                intersecting_sets.append(key)
            elif len_intersection==max_intersect_len:
                intersecting_sets.append(key)
                count_max+=1
        if max_intersect_len<=1:
            bucket_map[count_buckets] = keywords_set
            author_bucket_map[author_id] = count_buckets
            count_buckets+=1
        else:
            if count_max==1:
                bucket_map[intersecting_sets[0]] = bucket_map[intersecting_sets[0]] | keywords_set
                author_bucket_map[author_id] = intersecting_sets[0]
            else:
                min_size_bucket_id = -1
                min_size_buckets_count = 0
                min_size = sys.maxsize
                for intersecting_set in intersecting_sets:
                    size = len(bucket_map[intersecting_set])
                    if size==min_size:
                        min_size_buckets_count+=1
                    elif size<min_size:
                        min_size=size
                        min_size_bucket_id = intersecting_set
                        min_size_buckets_count=1
                bucket_map[min_size_bucket_id] = bucket_map[min_size_bucket_id] | keywords_set
                author_bucket_map[author_id] = min_size_bucket_id

In [95]:
final_buckets_list = [111, 122, 3, 138, 228, 946, 294, 85, 54, 168, 159, 81, 118, 344]
final_buckets_map = {111:-1, 122:-2, 3:-3, 138:-4, 228:-5, 946:-6, 294:-7, 85:-8, 54:-9, 168:-10, 159:-11, 81:-12, 118:-13, 344:-14}


In [27]:

def get_bucket_author_count_map(author_bucket_map):
    bucket_author_count_map = {}
    bucket_author_map = {}
    for key in author_bucket_map:
        value = author_bucket_map[key]
        if value in bucket_author_count_map:
            bucket_author_count_map[value]+=1
        else:
            bucket_author_count_map[value] = 1
            bucket_author_map[value] = set()
        bucket_author_map[value].add(key)
    return sorted(bucket_author_count_map.items(), key=lambda x: x[1], reverse=True ), bucket_author_map


In [59]:

def re_order_map(author_bucket_map, bucket_map, author_keyword_map, bucket_author_map, count_buckets):
    idx=0
    for bucket_id in bucket_author_map:
        if bucket_id in final_buckets_list:
            continue
        for author_id in bucket_author_map[bucket_id]:
            max_intersect_len = 0
            count_max = 0
            intersecting_sets = []
            bucket_map.pop(author_bucket_map[author_id], None)
            #print(author_bucket_map[author_id])
            keywords_set = author_keyword_map[author_id]
            for bucket_key in bucket_map:
                if bucket_key not in final_buckets_list:
                    continue
                len_intersection = len(keywords_set.intersection(bucket_map[bucket_key]))
                if len_intersection>max_intersect_len:
                    max_intersect_len = len_intersection
                    count_max = 1
                    intersecting_sets = []
                    intersecting_sets.append(bucket_key)
                elif len_intersection==max_intersect_len:
                    intersecting_sets.append(bucket_key)
                    count_max+=1
            if max_intersect_len<1:
                continue
#                 bucket_map[count_buckets] = keywords_set
#                 author_bucket_map[author_id] = count_buckets
#                 count_buckets+=1
            else:
                if count_max==1:
                    bucket_map[intersecting_sets[0]] = bucket_map[intersecting_sets[0]] | keywords_set
                    author_bucket_map[author_id] = intersecting_sets[0]
#                     print("----")
#                     print(author_id, bucket_map[intersecting_sets[0]])
                else:
                    min_size_bucket_id = -1
                    min_size_buckets_count = 0
                    min_size = sys.maxsize
                    for intersecting_set in intersecting_sets:
                        size = len(bucket_map[intersecting_set])
                        if size==min_size:
                            min_size_buckets_count+=1
                        elif size<min_size:
                            min_size=size
                            min_size_bucket_id = intersecting_set
                            min_size_buckets_count=1
                    bucket_map[min_size_bucket_id] = bucket_map[min_size_bucket_id] | keywords_set
                    author_bucket_map[author_id] = min_size_bucket_id
    return count_buckets, bucket_map, author_bucket_map



In [62]:
bucket_author_count_map, bucket_author_map = get_bucket_author_count_map(author_bucket_map)
while(len(bucket_author_count_map)>1000):
    count_buckets, bucket_map, author_bucket_map = re_order_map(author_bucket_map, bucket_map, author_keyword_map, bucket_author_map, count_buckets)
    bucket_author_count_map, bucket_author_map = get_bucket_author_count_map(author_bucket_map)
    print(len(bucket_author_count_map))

2285
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284
2284


KeyboardInterrupt: 

In [61]:

print(bucket_author_count_map)

[(111, 33495), (122, 32885), (3, 10224), (138, 2444), (228, 2224), (946, 2016), (294, 1944), (85, 1892), (54, 1794), (168, 1790), (159, 1783), (81, 1783), (118, 1710), (344, 1545), (346, 3), (2247, 2), (2361, 2), (2484, 2), (4882, 2), (4889, 2), (5849, 2), (5984, 2), (11199, 2), (11427, 2), (11531, 2), (13141, 2), (13452, 2), (13663, 2), (13805, 2), (14052, 2), (14263, 2), (14279, 2), (14720, 2), (15034, 2), (15116, 2), (15304, 2), (16205, 2), (17058, 2), (20207, 2), (20785, 2), (21600, 1), (21606, 1), (21610, 1), (21613, 1), (21614, 1), (21615, 1), (21619, 1), (21623, 1), (21624, 1), (21625, 1), (21626, 1), (21627, 1), (21628, 1), (21631, 1), (21635, 1), (21640, 1), (21644, 1), (21647, 1), (21650, 1), (21652, 1), (21653, 1), (21660, 1), (21662, 1), (21664, 1), (21669, 1), (21675, 1), (576, 1), (21680, 1), (21683, 1), (21687, 1), (21690, 1), (21693, 1), (21694, 1), (21699, 1), (21703, 1), (21704, 1), (21706, 1), (21707, 1), (21708, 1), (21713, 1), (21714, 1), (21720, 1), (21723, 1), (2

In [82]:
def print_h_index(bucket_author_map, authors_df, final_buckets_list):
    idx = 0
    for index, row in authors_df.iterrows():
        if row['author_id'] in bucket_author_map and bucket_author_map[row['author_id']] not in final_buckets_list: 
            print(row['author_id'], row['h_index'])
        idx+=1
        if idx==100000:
            break;

print_h_index(bucket_author_map, authors_df, final_buckets_list)

In [76]:
idx=0
for key in bucket_author_count_map:
    if idx==4:
        print(key[0], bucket_map[key[0]])
        print("=============================================================")
    if idx==10:
        break
    idx+=1

228 {'Core Ontological Model', 'reorder buffer', 'diabetes educator', 'Active Harmony system', 'current policy evaluation engine', 'square root', 'dividable large data task', 'protocol processing', 'VM Exit forwarding', 'massive data processing', 'tone acetyltransferase', 'physiological index', 'multi-cycle path', 'single collection event', 'engineering PSEs', 'hardware system', 'real engine control application', 'Blue GeneWe', 'current disk system', 'performance demand', 'associated website', 'imperfect load information', 'SBF algorithm', 'architecturesAnalyzing parallel program performance', 'customer issue', 'server side data', 'efficient address translation', 'CDN energy simulator', 'average error', 'FORTRAN IV computer program', 'object code size elimination', 'GPU virtualization', 'European Grid Initiative platform', 'efficient file access mechanism', 'accessing user information', 'novel user-level tool', 'different communication scenario', 'multi-core CPU', 'SIMD array', 'curren

In [79]:
for bucket in bucket_map:
    if bucket in final_buckets_list:
        print(bucket, "----------------------------------------------------------------------------------------------------------------")
        print(bucket_map[bucket])
        print("----------------------------------------------------------------------------------------------------------------")

3 ----------------------------------------------------------------------------------------------------------------
{'Autodesk Inventor LT', 'nonrigid image registration technique', 'Multi-path PALSAR interferometric observation', 'kernel principal component analysis', 'numerosity-constrained random networksIn', 'Interlingua Representation', 'geo data cube', 'color concentration', 'point-sampled surface model', 'Web-based robotic telecare system', 'Observation Model', 'Matrix-Vector Multiplication', 'normal gait study', 'feature point search', 'local image phase', 'data hider', 'average evacuation time', 'rule-based system', 'assembly cost', 'empirical ambient occlusion technique', 'Information Studies', 'commercial vessel', 'physiological index', 'vehicle localizationFor', 'mixed reality visualization', 'progressive video sequence', 'realistic expression', 'global image space', 'Al strip', 'noise property', 'arbitrary digital contour', 'popular approach', 'compound classification', 'gl

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [93]:
words_count = {}

for bucket in final_buckets_list:
    word_count = {}
    for keyword in bucket_map[bucket]:
        word_arr = keyword.split(' ')
        for word in word_arr:
            if word in word_count:
                word_count[word] = word_count[word]+1
            else:
                word_count[word] = 1
        words_count[bucket] = word_count
    

In [94]:

for key in words_count:
    idx = 0
    d = words_count[key]
    print("key = ", key)
    sorted_word_count = [(k, d[k]) for k in sorted(d, key=d.get, reverse=True)]
    for k, v in sorted_word_count:
        print(k, v)
        idx+=1
        if idx==15:
            break
    

key =  111
data 9097
system 6755
model 4871
information 3197
algorithm 2495
method 2386
network 2107
analysis 1988
problem 1825
learning 1738
process 1711
language 1668
service 1658
design 1602
new 1585
key =  122
system 7328
network 6386
model 4446
algorithm 3329
control 3057
data 2941
problem 2328
method 2318
design 2052
service 2012
information 1850
scheme 1829
power 1792
new 1779
performance 1663
key =  3
image 3992
model 1861
system 1783
method 1696
algorithm 1344
data 1232
motion 995
video 967
segmentation 721
detection 702
feature 696
recognition 696
new 661
object 652
shape 639
key =  138
software 1726
model 618
system 574
process 438
development 411
engineering 315
product 292
design 241
service 236
requirements 235
data 198
Web 193
testing 185
project 178
approach 175
key =  228
performance 1103
memory 701
system 647
data 387
cache 357
model 266
application 260
algorithm 250
parallel 217
code 189
analysis 177
network 172
architecture 163
hardware 154
processor 152
key =  946


In [96]:
def add_cluster(row, author_bucket_map):
    if row['author_id'] in author_bucket_map:
        value = author_bucket_map[row['author_id']]
        if value in final_buckets_map:
            return final_buckets_map[value]
    return 9999999

authors_df['cluster_id'] = authors_df.apply(lambda x: add_cluster(x, author_bucket_map), axis=1)


In [97]:
authors_df.to_csv('./data/authors_with_clusters_final.csv', sep=',')
authors_df.head(1000)

,Unnamed: 0,author_id,citations,co_authors,h_index,num_citations,paper_count,keywords,affiliation,cluster_id
40064,Jiawei_Han,40255,95502,"[42167, 11017, 44161, 16038, 64503, 64504, 645...",144,29666.590508,737,proposed method;Problem Determination;computer...,"Department of Neurosurgery, University Hospita...",-2
45068,Anil_Jain,45287,121854,"[45286, 66718, 66719, 67047, 67048, 67049, 787...",144,49936.934754,549,computer vision;Feature Measurement;complete v...,"Department of Mathematics, Wuyi University, Ji...",-2
736,Philip_Yu,738,89103,"[2492, 9499, 9500, 13689, 13690, 24400, 16425,...",132,27231.563889,976,ranking data;weighted distance-based model;GAR...,Department of Statistics and Actuarial Science...,-1
6993,Andrew_Zisserman,7015,78278,"[5776, 46602, 96985, 31335, 81469, 100220, 100...",123,25981.066598,360,image pair;query image;real image;real image s...,"Univ. of Edinburgh, Edinburgh, Scotland;Oxford...",-3
15790,Thomas_Huang,15850,79560,"[15849, 27280, 27281, 45764, 82995, 88292, 640...",122,24454.486760,827,facial component;facial contour;facial feature...,"University of Illinois at Urbana Champaign, Ur...",-3
51275,Scott_Shenker,51523,96949,"[51524, 7910, 55012, 31553, 55013, 26096, 8558...",122,20813.985300,269,low duty cycle;Asynchronous Neighbor Discovery...,"Université, ParisXI, France;Université ParisSu...",-2
29231,Hector_Garcia-Molina,29354,61993,"[13989, 29355, 46601, 24848, 56891, 28727, 568...",117,22743.585762,400,data model;data warehousing;data cloud;data it...,"Princeton Univ., Princeton, NJ;Princeton;Arvin...",-1
25717,Sebastian_Thrun,25821,61045,"[25815, 25816, 25817, 25818, 25819, 25820, 668...",113,23253.652814,268,mobile robot;real robot;mobile robot navigatio...,"Carnegie Mellon Univ., Pittsburgh, PA;Computer...",-11
78068,Michael_Jordan,78466,82293,"[52563, 80489, 94259, 4836, 94260, 4822, 94261...",113,26553.652020,334,supply chain management;interorganisational in...,"Caledonian Business School, Glasgow Caledonian...",-1
22201,Christos_Faloutsos,22285,70392,"[22283, 22284, 38001, 38002, 38003, 41734, 537...",112,22553.027524,512,social network;online social network;connected...,Carnegie Mellon University,-1


In [ ]:
all_keywords = []
useless_ketwoeds = ['local', 'based', 'approach', 'user', 'environment', 'proposed', 'method', 'proposed',
                   'technique', 'framework', ]
for index, row in authors_df.iterrows():
    if not pd.isna(row['keywords']):
        keywords = row['keywords'].split(';')
        for keyword in keywords:
            all_keywords.append(keyword)

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(all_keywords)
true_k = 20
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])
    print()

In [ ]:
for i in range(true_k):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :25]:
        print(' %s' % terms[ind])
    print()

In [98]:
#reading the training data and test data

file_name = 'dblp-ref-'
num_files = 4
train_data = []

for i in range(num_files):
    train_data.append(pd.read_json("./data/dblp-ref/dblp-ref-" + str(i) + ".json", lines=2))
    train_data[i].head(3)


In [104]:
#authors_df.rename( columns={'Unnamed: 0':'author_name'}, inplace=True )
authors_df.head()

,author_name,author_id,citations,co_authors,h_index,num_citations,paper_count,keywords,affiliation,cluster_id
40064,Jiawei_Han,40255,95502,"[42167, 11017, 44161, 16038, 64503, 64504, 645...",144,29666.590508,737,proposed method;Problem Determination;computer...,"Department of Neurosurgery, University Hospita...",-2
45068,Anil_Jain,45287,121854,"[45286, 66718, 66719, 67047, 67048, 67049, 787...",144,49936.934754,549,computer vision;Feature Measurement;complete v...,"Department of Mathematics, Wuyi University, Ji...",-2
736,Philip_Yu,738,89103,"[2492, 9499, 9500, 13689, 13690, 24400, 16425,...",132,27231.563889,976,ranking data;weighted distance-based model;GAR...,Department of Statistics and Actuarial Science...,-1
6993,Andrew_Zisserman,7015,78278,"[5776, 46602, 96985, 31335, 81469, 100220, 100...",123,25981.066598,360,image pair;query image;real image;real image s...,"Univ. of Edinburgh, Edinburgh, Scotland;Oxford...",-3
15790,Thomas_Huang,15850,79560,"[15849, 27280, 27281, 45764, 82995, 88292, 640...",122,24454.486760,827,facial component;facial contour;facial feature...,"University of Illinois at Urbana Champaign, Ur...",-3


In [100]:
# authors_df.columns = ['author_name', 'author_id' 'citations','co_authors','h_index', 
#                       'num_citations','paper_count', 'keywords', 'affiliation', 'cluster_id']
# authors_df.describe()

ValueError: Length mismatch: Expected axis has 10 elements, new values have 9 elements

In [105]:
author_name_cluster_map = {}
for index, row in authors_df.iterrows():
    name = row['author_name']
    if name not in author_name_cluster_map:
        author_name_cluster_map [name] = row['cluster_id']


In [107]:
train_data[0].head()

,abstract,authors,id,n_citation,references,title,venue,year
0,The purpose of this study is to develop a lear...,"[Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Ka...",00127ee2-cb05-48ce-bc49-9de556b93346,0,"[51c7e02e-f5ed-431a-8cf5-f761f266d4be, 69b625b...",Preliminary Design of a Network Protocol Learn...,international conference on human-computer int...,2013
1,This paper describes the design and implementa...,"[Gareth Beale, Graeme Earl]",001c58d3-26ad-46b3-ab3a-c1e557d16821,50,"[10482dd3-4642-4193-842f-85f3b70fcf65, 3133714...",A methodology for the physically accurate visu...,visual analytics science and technology,2011
2,This article applied GARCH model instead AR or...,"[Altaf Hossain, Faisal Zaman, Mohammed Nasser,...",001c8744-73c4-4b04-9364-22d31a10dbf1,50,"[2d84c0f2-e656-4ce7-b018-90eda1c132fe, a083a1b...","Comparison of GARCH, Neural Network and Suppor...",pattern recognition and machine intelligence,2009
3,NaN,"[Jea-Bum Park, Byungmok Kim, Jian Shen, Sun-Yo...",00338203-9eb3-40c5-9f31-cbac73a519ec,0,"[8c78e4b0-632b-4293-b491-85b1976675e6, 9cdc54f...",Development of Remote Monitoring and Control D...,,2011
4,NaN,"[Giovanna Guerrini, Isabella Merlo]",0040b022-1472-4f70-a753-74832df65266,2,NaN,Reasonig about Set-Oriented Methods in Object ...,,1998


In [ ]:
def get_paper_cluster(x, author_name_cluster_map):
    authors = row['authors'].replace("[", "").replace("]", "").split(",")
    for author in authors:
        if author in author_name_cluster_map:
            

for i in range(4):
    train_data['cluster_id'] = train_data[i].apply(lambda x: get_paper_cluster(x, author_name_cluster_map), axis=1)